# Perceptron Implementation on Penguins Dataset
This notebook demonstrates binary classification using the Perceptron algorithm.
We classify between **Adelie** and **Gentoo** penguins using four numeric features.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Load and clean the data
penguins_df = pd.read_csv("penguins.csv")
penguins_df_clean = penguins_df.dropna()
penguins_df_binary = penguins_df_clean[penguins_df_clean['species'].isin(['Adelie', 'Gentoo'])]
penguins_df_binary['label'] = penguins_df_binary['species'].map({'Adelie': 0, 'Gentoo': 1})

In [ ]:
# Feature selection
features = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
X = penguins_df_binary[features].values
y = penguins_df_binary['label'].values

In [ ]:
# Normalize and add bias
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X_norm = (X - X_mean) / X_std
X_bias = np.c_[np.ones(X_norm.shape[0]), X_norm]

In [ ]:
# Perceptron training function
def perceptron_train(X, y, epochs=10):
    weights = np.zeros(X.shape[1])
    for epoch in range(epochs):
        for i in range(X.shape[0]):
            prediction = 1 if np.dot(weights, X[i]) > 0 else 0
            error = y[i] - prediction
            weights += error * X[i]
    return weights

final_weights = perceptron_train(X_bias, y, epochs=10)

In [ ]:
# Prediction function and accuracy
def perceptron_predict(X, weights):
    return np.where(np.dot(X, weights) > 0, 1, 0)

predictions = perceptron_predict(X_bias, final_weights)
accuracy = np.mean(predictions == y)
print(f"Accuracy: {accuracy:.2f}")